In [19]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
#import psycopg2

# from config import db_password

import time

In [20]:
# 1. Add the clean movie function that takes in the argument, "movie".

# Add function from this module for clean movie
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    alt_titles = {}
    # combine alternate titles into one list
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    return movie
    
    
    

 

In [21]:
def parse_dollars(s):
    if type(s) != str:
        return np.nan
    if s == '':
        return np.nan
    s = s.replace('$', '').replace(',','').lower()
    # if input is of the form ###.# mil
    p = re.compile(r'\d+\.?\d*\s*mil')
    match = re.search(p, s)
    if match:
        total_box_office = match.group(0).replace('mil', '').rstrip()
        value = float(total_box_office) * 10**6
        return value
    p = re.compile(r'\d+\.?\d*\s*bil')
    match = re.search(p, s)
    if match:
        total_box_office = match.group().replace('bil', '').rstrip()
        value = float(total_box_office) * 10**9
        return value
    p = re.compile(r'\d+')
    match = re.search(p, s)
    if match:
        total_box_office = match.group()
        value = float(total_box_office)
        return value
    return np.nan

In [22]:
def parse_dates(s):
    if type(s) != str:
        return np.nan
    if s == '':
        return np.nan
    s = s.replace('$', '').replace(',','').lower()
    # if input is of the form ###.# mil
   
    p = re.compile(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}')
    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value
    
    p = re.compile(r'\d+\.?\d*\s*bil')
    match = re.search(p, s)
    if match:
        value = match.group(0)
        return value
   
    p = re.compile(r'\d{4}.[01]\d.[123]\d')
    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value
     
    p = re.compile (r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}')
    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value
     
    p = re.compile (r'\d{4}')

    match = re.search(p, s)
    if match:
    
        value = match.group(0)
        return value

    return np.nan

In [23]:
def parse_times(s):
    if type(s) != str:
        return np.nan
    if s == '':
        return np.nan
    p = re.compile(r'\d* min')
    match = re.search(p, s)
    if match:
        value = match.group(0)
        value = value.replace(' min', '')
        #value = int(value)
        return value
    return np.nan

In [24]:
file_dir = 'C:/users/lrosa/OneDrive/Desktop/Module 8'
kaggle_metadata=(f"{file_dir}/movies_metadata.csv")

ratings=(f"{file_dir}/ratings.csv")

wiki_movies_raw=(f'{file_dir}/wikipedia-movies.json') 

In [25]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)


 #Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.


# Open and read the Wikipedia data JSON file.

In [26]:
def extract_transform_load(kaggle_metadata,ratings,wiki_movies_raw):
    kaggle_metadata=pd.read_csv(kaggle_metadata,low_memory=False)

    ratings=pd.read_csv(ratings,low_memory=False)

    with open(wiki_movies_raw, mode='r') as file:
        wiki_movies_raw = json.load(file)





    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies=[movie for movie in wiki_movies_raw if("Director" in movie or "Directed by" in movie) \
             and "imdb_link" in movie and "No. of episodes" not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies=[clean_movie(movie)for movie in wiki_movies]
    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df=pd.DataFrame(clean_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    # extract IMDb ID
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    cols=[column for column in wiki_movies_df.columns if wiki_movies_df[column].count()>(wiki_movies_df.shape[0]/2)]
    wiki_movies_df=wiki_movies_df[cols]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office=wiki_movies_df["Box office"].dropna()

    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    r'\$\d+\.?\d*\s*mil.*'
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    # 12. Add the parse_dollars function.
    box_office=box_office.apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    wiki_movies_df['Box office'] = box_office
    wiki_movies_df['Box office']
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    Budget_office=wiki_movies_df["Budget"].dropna()
    Budget_office = Budget_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
    Budget_office=Budget_office.apply(parse_dollars)
    wiki_movies_df.drop("Budget",axis=1, inplace= True)
    wiki_movies_df["Budget"]=Budget_office
    # 15. Clean the release date column in the wiki_movies_df DataFrame.


    Release=wiki_movies_df["Release date"].dropna()
    Release = Release.apply(lambda x: ' '.join(x) if type(x) == list else x)
    Release =Release.apply(parse_dates)
    wiki_movies_df.drop("Release date",axis=1, inplace= True)
    wiki_movies_df["Release date"]=Release
    wiki_movies_df
    # 16. Clean the running time column in the wiki_movies_df DataFrame.


    #16 Clean the running time column in the wiki_movies_df DataFrame
    running_time = wiki_movies_df['Running time'].dropna()
    running_time = running_time.apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time = running_time.apply(parse_times)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    wiki_movies_df['Running time'] = running_time
    
    return kaggle_metadata,ratings,wiki_movies_df

In [28]:
file_dir = 'Resources'
wiki_file = f'{file_dir}/wikipedia-movies.json'
kaggle_file = f'{file_dir}/movies_metadata.csv'
ratings_file = f'{file_dir}/ratings_small.csv'
kaggle_df, ratings_df,wiki_df = extract_transform_load(kaggle_file, ratings_file, wiki_file)

In [29]:
wiki_df

url  year  \
0     https://en.wikipedia.org/wiki/The_Adventures_o...  1990   
1     https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990   
2      https://en.wikipedia.org/wiki/Air_America_(film)  1990   
3       https://en.wikipedia.org/wiki/Alice_(1990_film)  1990   
4         https://en.wikipedia.org/wiki/Almost_an_Angel  1990   
...                                                 ...   ...   
7071    https://en.wikipedia.org/wiki/Holmes_%26_Watson  2018   
7072     https://en.wikipedia.org/wiki/Vice_(2018_film)  2018   
7073  https://en.wikipedia.org/wiki/On_the_Basis_of_Sex  2018   
7074  https://en.wikipedia.org/wiki/Destroyer_(2018_...  2018   
7075  https://en.wikipedia.org/wiki/Black_Mirror:_Ba...  2018   

                                  imdb_link                            title  \
0     https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
1     https://www.imdb.com/title/tt0098994/             After Dark, My Sweet   
2     https://www.imdb.com/title/tt0099005/                      Air America   
3     https://www.imdb.com/title/tt0099012/                            Alice   
4     https://www.imdb.com/title/tt0099018/                  Almost an Angel   
...                                     ...                              ...   
7071  https://www.imdb.com/title/tt1255919/                  Holmes & Watson   
7072  https://www.imdb.com/title/tt6266538/                             Vice   
7073  https://www.imdb.com/title/tt4669788/              On the Basis of Sex   
7074  https://www.imdb.com/title/tt7137380/                        Destroyer   
7075  https://www.imdb.com/title/tt9495224/                     Bandersnatch   

                                               Starring  \
0     [Andrew Dice Clay, Wayne Newton, Priscilla Pre...   
1     [Jason Patric, Rachel Ward, Bruce Dern, George...   
2     [Mel Gibson, Robert Downey Jr., Nancy Travis, ...   
3     [Alec Baldwin, Blythe Danner, Judy Davis, Mia ...   
4           [Paul Hogan, Elias Koteas, Linda Kozlowski]   
...                                                 ...   
7071  [Will Ferrell, John C. Reilly, Rebecca Hall, R...   
7072  [Christian Bale, Amy Adams, Steve Carell, Sam ...   
7073  [Felicity Jones, Armie Hammer, Justin Theroux,...   
7074  [Nicole Kidman, Sebastian Stan, Toby Kebbell, ...   
7075  [Fionn Whitehead, Will Poulter, Craig Parkinso...   

                     Cinematography         Country        Language  \
0                       Oliver Wood   United States         English   
1                      Mark Plummer   United States         English   
2                     Roger Deakins   United States  [English, Lao]   
3                    Carlo Di Palma   United States         English   
4                      Russell Boyd              US         English   
...                             ...             ...             ...   
7071                    Oliver Wood   United States         English   
7072                   Greig Fraser   United States         English   
7073                  Michael Grady   United States         English   
7074                 Julie Kirkwood   United States         English   
7075  [Aaron Morton, Jake Polonsky]  United Kingdom         English   

                Director              Distributor  \
0           Renny Harlin         20th Century Fox   
1            James Foley          Avenue Pictures   
2     Roger Spottiswoode         TriStar Pictures   
3            Woody Allen           Orion Pictures   
4           John Cornell       Paramount Pictures   
...                  ...                      ...   
7071          Etan Cohen  Sony Pictures Releasing   
7072          Adam McKay         Mirror Releasing   
7073          Mimi Leder           Focus Features   
7074        Karyn Kusama         Mirror Releasing   
7075         David Slade                  Netflix   

                           Editor(s)              Composer(s)  \
0                    Michael Tronick  [Cliff E

adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                                  genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                         [{'id': 35, 'name': 'Comedy'}]   
...                                                  ...   
45461  [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...   
45462                      [{'id': 18, 'name': 'Drama'}]   
45463  [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...   
45464                                                 []   
45465                                                 []   

                                   homepage      id    imdb_id  \
0      http://toystory.disney.com/toy-story     862  tt0114709   
1                                       NaN    8844  tt0113497   
2                                       NaN   15602  tt0113228   
3                                       NaN   31357  tt0114885   
4                                       NaN   11862  tt0113041   
...                                     ...     ...        ...   
45461  http://www.imdb.com/title/tt6209470/  439050  tt6209470   
45462                                   NaN  111109  tt2028550   
45463                                   NaN   67758  tt0303758   
45464                                   NaN  227506  tt0008536   
45465                                   NaN  461257  tt6980792   

      original_language               original_title  \
0                    en                    Toy Story   
1                    en                      Jumanji   
2                    en             Grumpier Old Men   
3                    en            Waiting to Exhale   
4                    en  Father of the Bride Part II   
...                 ...                          ...   
45461                fa                      رگ خواب   
45462                tl          Siglo ng Pagluluwal   
45463                en                     Betrayal   
45464                en          Satana likuyushchiy   
45465                en                     Queerama   

                                                overview  ... release_date  \
0      Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1      When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2      A family wedding reignites the ancient feud be...  ...   1995-12-22   
3      Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4      Just when George Banks has recovered from his ...  ...   1995-02-10   
...                                                  ...  ...          ...   
45461        Rising and falling between a man and woman.  ...          NaN   
45462  An artist struggles to finish his work while a...  ...   2011-11-17   
45463  When one of her hits goes wrong, a professiona...  ...   2003-08-01   
45464  In a small town live two brothers, one a minis...  ...   1917-10-21   
45465  50 years after decriminalisati

userId  movieId  rating   timestamp
0            1       31     2.5  1260759144
1            1     1029     3.0  1260759179
2            1     1061     3.0  1260759182
3            1     1129     2.0  1260759185
4            1     1172     4.0  1260759205
...        ...      ...     ...         ...
99999      671     6268     2.5  1065579370
100000     671     6269     4.0  1065149201
100001     671     6365     4.0  1070940363
100002     671     6385     2.5  1070979663
100003     671     6565     3.5  1074784724

[100004 rows x 4 columns]

In [33]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_df

url  year  \
0     https://en.wikipedia.org/wiki/The_Adventures_o...  1990   
1     https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990   
2      https://en.wikipedia.org/wiki/Air_America_(film)  1990   
3       https://en.wikipedia.org/wiki/Alice_(1990_film)  1990   
4         https://en.wikipedia.org/wiki/Almost_an_Angel  1990   
...                                                 ...   ...   
7071    https://en.wikipedia.org/wiki/Holmes_%26_Watson  2018   
7072     https://en.wikipedia.org/wiki/Vice_(2018_film)  2018   
7073  https://en.wikipedia.org/wiki/On_the_Basis_of_Sex  2018   
7074  https://en.wikipedia.org/wiki/Destroyer_(2018_...  2018   
7075  https://en.wikipedia.org/wiki/Black_Mirror:_Ba...  2018   

                                  imdb_link                            title  \
0     https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
1     https://www.imdb.com/title/tt0098994/             After Dark, My Sweet   
2     https://www.imdb.com/title/tt0099005/                      Air America   
3     https://www.imdb.com/title/tt0099012/                            Alice   
4     https://www.imdb.com/title/tt0099018/                  Almost an Angel   
...                                     ...                              ...   
7071  https://www.imdb.com/title/tt1255919/                  Holmes & Watson   
7072  https://www.imdb.com/title/tt6266538/                             Vice   
7073  https://www.imdb.com/title/tt4669788/              On the Basis of Sex   
7074  https://www.imdb.com/title/tt7137380/                        Destroyer   
7075  https://www.imdb.com/title/tt9495224/                     Bandersnatch   

                                               Starring  \
0     [Andrew Dice Clay, Wayne Newton, Priscilla Pre...   
1     [Jason Patric, Rachel Ward, Bruce Dern, George...   
2     [Mel Gibson, Robert Downey Jr., Nancy Travis, ...   
3     [Alec Baldwin, Blythe Danner, Judy Davis, Mia ...   
4           [Paul Hogan, Elias Koteas, Linda Kozlowski]   
...                                                 ...   
7071  [Will Ferrell, John C. Reilly, Rebecca Hall, R...   
7072  [Christian Bale, Amy Adams, Steve Carell, Sam ...   
7073  [Felicity Jones, Armie Hammer, Justin Theroux,...   
7074  [Nicole Kidman, Sebastian Stan, Toby Kebbell, ...   
7075  [Fionn Whitehead, Will Poulter, Craig Parkinso...   

                     Cinematography         Country        Language  \
0                       Oliver Wood   United States         English   
1                      Mark Plummer   United States         English   
2                     Roger Deakins   United States  [English, Lao]   
3                    Carlo Di Palma   United States         English   
4                      Russell Boyd              US         English   
...                             ...             ...             ...   
7071                    Oliver Wood   United States         English   
7072                   Greig Fraser   United States         English   
7073                  Michael Grady   United States         English   
7074                 Julie Kirkwood   United States         English   
7075  [Aaron Morton, Jake Polonsky]  United Kingdom         English   

                Director              Distributor  \
0           Renny Harlin         20th Century Fox   
1            James Foley          Avenue Pictures   
2     Roger Spottiswoode         TriStar Pictures   
3            Woody Allen           Orion Pictures   
4           John Cornell       Paramount Pictures   
...                  ...                      ...   
7071          Etan Cohen  Sony Pictures Releasing   
7072          Adam McKay         Mirror Releasing   
7073          Mimi Leder           Focus Features   
7074        Karyn Kusama         Mirror Releasing   
7075         David Slade                  Netflix   

                           Editor(s)              Composer(s)  \
0                    Michael Tronick  [Cliff E

In [34]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Starring',
 'Cinematography',
 'Country',
 'Language',
 'Director',
 'Distributor',
 'Editor(s)',
 'Composer(s)',
 'Producer(s)',
 'Production company(s)',
 'Writer(s)',
 'imdb_id',
 'Box office',
 'Budget',
 'Release date',
 'Running time']

In [35]:
kaggle_df = kaggle_df[kaggle_df['adult'] == 'False'].drop('adult',axis='columns')
kaggle_df['video'] = kaggle_df['video'] == 'True'
kaggle_df['budget'] = kaggle_df['budget'].astype(int)
kaggle_df['id'] = pd.to_numeric(kaggle_df['id'], errors='raise')
kaggle_df['popularity'] = pd.to_numeric(kaggle_df['popularity'], errors='raise')
kaggle_df['release_date'] = pd.to_datetime(kaggle_df['release_date'], errors='raise')

In [36]:
# Merge the wiki and kaggle data
movies_df = pd.merge(wiki_df, kaggle_df, on='imdb_id', suffixes=['_wiki','_kaggle'])

In [37]:
# drop unnecessary columns from the movies_df DataFrame
movies_df.drop(columns=[
    'spoken_languages', 'poster_path', 'Production company(s)'], 
    inplace=True)

In [38]:
# Step 5, add the fill_missing_kaggle_data() function that fills in the missing Kaggle data on the movies_df DataFrame.
def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
    df[kaggle_column] = df.apply(
        lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
        , axis=1)
    df.drop(columns=wiki_column, inplace=True)

In [39]:
# In Step 6, call the fill_missing_kaggle_data() function with the movies_df DataFrame and the Kaggle and Wikipedia columns to be cleaned as the arguments
fill_missing_kaggle_data(movies_df, 'runtime', 'Running time')
fill_missing_kaggle_data(movies_df, 'budget', 'Budget')
fill_missing_kaggle_data(movies_df, 'revenue', 'Box office')

In [40]:
# In Step 7, filter the movies_df DataFrame to keep the necessary columns.
movies_df = movies_df[['imdb_id','id','title_kaggle','original_title','tagline',
                       'belongs_to_collection','url','imdb_link','runtime',
                       'revenue','popularity','vote_average','vote_count','genres',
                       'original_language','overview','Country','production_companies',
                       'production_countries','Distributor','Producer(s)','Director',
                       'Starring','Cinematography','Editor(s)','Writer(s)',
                       'Composer(s)']]

In [41]:
# In Step 8, rename the columns in the movies_df DataFrame.
movies_df.rename({'id':'kaggle_id',
                'title_kaggle':'title',
                'url':'wikipedia_url',
                'Country':'country',
                'Distributor':'distributor',
                'Producer(s)':'producers',
                'Director':'director',
                'Starring':'starring',
                'Cinematography':'cinematography',
                'Editor(s)':'editors',
                'Writer(s)':'writers',
                'Composer(s)':'composers',
                }, axis='columns', inplace=True)

In [42]:
# In Step 9, transform and merge the ratings DataFrame with the movies_df DataFrame, 
# name the new DataFrame movies_with_ratings_df, then clean the movies_with_ratings_df DataFrame
rating_counts = ratings_df.groupby(['movieId','rating'], as_index=False).count()
rating_counts = rating_counts.rename({'userId':'count'}, axis=1)
rating_counts = rating_counts.pivot(index='movieId',columns='rating', values='count')
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
movies_with_ratings_df = pd.merge(
    movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

In [45]:
##step 13
wiki_df.head(5)

url  year  \
0  https://en.wikipedia.org/wiki/The_Adventures_o...  1990   
1  https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990   
2   https://en.wikipedia.org/wiki/Air_America_(film)  1990   
3    https://en.wikipedia.org/wiki/Alice_(1990_film)  1990   
4      https://en.wikipedia.org/wiki/Almost_an_Angel  1990   

                               imdb_link                            title  \
0  https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
1  https://www.imdb.com/title/tt0098994/             After Dark, My Sweet   
2  https://www.imdb.com/title/tt0099005/                      Air America   
3  https://www.imdb.com/title/tt0099012/                            Alice   
4  https://www.imdb.com/title/tt0099018/                  Almost an Angel   

                                            Starring  Cinematography  \
0  [Andrew Dice Clay, Wayne Newton, Priscilla Pre...     Oliver Wood   
1  [Jason Patric, Rachel Ward, Bruce Dern, George...    Mark Plummer   
2  [Mel Gibson, Robert Downey Jr., Nancy Travis, ...   Roger Deakins   
3  [Alec Baldwin, Blythe Danner, Judy Davis, Mia ...  Carlo Di Palma   
4        [Paul Hogan, Elias Koteas, Linda Kozlowski]    Russell Boyd   

         Country        Language            Director         Distributor  \
0  United States         English        Renny Harlin    20th Century Fox   
1  United States         English         James Foley     Avenue Pictures   
2  United States  [English, Lao]  Roger Spottiswoode    TriStar Pictures   
3  United States         English         Woody Allen      Orion Pictures   
4             US         English        John Cornell  Paramount Pictures   

                        Editor(s)              Composer(s)  \
0                 Michael Tronick  [Cliff Eidelman, Yello]   
1                 Howard E. Smith            Maurice Jarre   
2  [John Bloom, Lois Freeman-Fox]            Charles Gross   
3                  Susan E. Morse                      NaN   
4                    David Stiven            Maurice Jarre   

                   Producer(s)                  Production company(s)  \
0   [Steve Perry, Joel Silver]                        Silver Pictures   
1  [Ric Kidney, Robert Redlin]                        Avenue Pictures   
2               Daniel Melnick  [Carolco Pictures, IndieProd Company]   
3              Robert Greenhut                                    NaN   
4                 John Cornell                                    NaN   

                      Writer(s)    imdb_id  Box office      Budget  \
0   [David Arnott, James Cappe]  tt0098987  21400000.0  20000000.0   
1  [James Foley, Robert Redlin]  tt0098994   2700000.0   6000000.0   
2    [John Eskow, Richard Rush]  tt0099005  57718089.0  35000000.0   
3                   Woody Allen  tt0099012   7331647.0  12000000.0   
4                    Paul Hogan  tt0099018   6939946.0  25000000.0   

  Release date Running time  
0   1990-07-11          102  
1   1990-05-17          114  
2   1990-08-10          113  
3   1990-12-25          106  
4         1990           95

In [43]:
#step 14
movies_df.head(5)

imdb_id  kaggle_id                            title  \
0  tt0098987       9548  The Adventures of Ford Fairlane   
1  tt0098994      25501             After Dark, My Sweet   
2  tt0099005      11856                      Air America   
3  tt0099012       8217                            Alice   
4  tt0099018      25943                  Almost an Angel   

                    original_title                                  tagline  \
0  The Adventures of Ford Fairlane      Kojak. Columbo. Dirty Harry. Wimps.   
1             After Dark, My Sweet          All they risked was everything.   
2                      Air America  The few. The proud. The totally insane.   
3                            Alice                                      NaN   
4                  Almost an Angel                 Who does he think he is?   

  belongs_to_collection                                      wikipedia_url  \
0                   NaN  https://en.wikipedia.org/wiki/The_Adventures_o...   
1                   NaN  https://en.wikipedia.org/wiki/After_Dark,_My_S...   
2                   NaN   https://en.wikipedia.org/wiki/Air_America_(film)   
3                   NaN    https://en.wikipedia.org/wiki/Alice_(1990_film)   
4                   NaN      https://en.wikipedia.org/wiki/Almost_an_Angel   

                               imdb_link runtime     revenue  ...  \
0  https://www.imdb.com/title/tt0098987/     104  20423389.0  ...   
1  https://www.imdb.com/title/tt0098994/     114   2700000.0  ...   
2  https://www.imdb.com/title/tt0099005/     112  33461269.0  ...   
3  https://www.imdb.com/title/tt0099012/     102   7331647.0  ...   
4  https://www.imdb.com/title/tt0099018/      95   6939946.0  ...   

                                production_companies  \
0  [{'name': 'Twentieth Century Fox Film Corporat...   
1  [{'name': 'Avenue Pictures Productions', 'id':...   
2  [{'name': 'IndieProd Company Productions', 'id...   
3             [{'name': 'Orion Pictures', 'id': 41}]   
4          [{'name': 'Paramount Pictures', 'id': 4}]   

                                production_countries         distributor  \
0  [{'iso_3166_1': 'US', 'name': 'United States o...    20th Century Fox   
1  [{'iso_3166_1': 'US', 'name': 'United States o...     Avenue Pictures   
2  [{'iso_3166_1': 'US', 'name': 'United States o...    TriStar Pictures   
3  [{'iso_3166_1': 'US', 'name': 'United States o...      Orion Pictures   
4  [{'iso_3166_1': 'US', 'name': 'United States o...  Paramount Pictures   

                     producers            director  \
0   [Steve Perry, Joel Silver]        Renny Harlin   
1  [Ric Kidney, Robert Redlin]         James Foley   
2               Daniel Melnick  Roger Spottiswoode   
3              Robert Greenhut         Woody Allen   
4                 John Cornell        John Cornell   

                                            starring  cinematography  \
0  [Andrew Dice Clay, Wayne Newton, Priscilla Pre...     Oliver Wood   
1  [Jason Patric, Rachel Ward, Bruce Dern, George...    Mark Plummer   
2  [Mel Gibson, Robert Downey Jr., Nancy Travis, ...   Roger Deakins   
3  [Alec Baldwin, Blythe Danner, Judy Davis, Mia ...  Carlo Di Palma   
4        [Paul Hogan, Elias Koteas, Linda Kozlowski]    Russell Boyd   

                          editors                       writers  \
0                 Michael Tronick   [David Arnott, James Cappe]   
1                 Howard E. Smith  [James Foley, Robert Redlin]   
2  [John Bloom, Lois Freeman-Fox]    [John Eskow, Richard Rush]   
3                  Susan E. Morse                   Woody Allen   
4                    David Stiven                    Paul Hogan   

                 composers  
0  [Cliff Eidelman, Yello]  
1            Maurice Jarre  
2            Charles Gross  
3                      NaN  
4            Maurice Jarre  

[5 rows x 27 columns]

In [44]:
#step 15
movies_with_ratings_df.head(5)

imdb_id  kaggle_id                            title  \
0  tt0098987       9548  The Adventures of Ford Fairlane   
1  tt0098994      25501             After Dark, My Sweet   
2  tt0099005      11856                      Air America   
3  tt0099012       8217                            Alice   
4  tt0099018      25943                  Almost an Angel   

                    original_title                                  tagline  \
0  The Adventures of Ford Fairlane      Kojak. Columbo. Dirty Harry. Wimps.   
1             After Dark, My Sweet          All they risked was everything.   
2                      Air America  The few. The proud. The totally insane.   
3                            Alice                                      NaN   
4                  Almost an Angel                 Who does he think he is?   

  belongs_to_collection                                      wikipedia_url  \
0                   NaN  https://en.wikipedia.org/wiki/The_Adventures_o...   
1                   NaN  https://en.wikipedia.org/wiki/After_Dark,_My_S...   
2                   NaN   https://en.wikipedia.org/wiki/Air_America_(film)   
3                   NaN    https://en.wikipedia.org/wiki/Alice_(1990_film)   
4                   NaN      https://en.wikipedia.org/wiki/Almost_an_Angel   

                               imdb_link runtime     revenue  ...  rating_0.5  \
0  https://www.imdb.com/title/tt0098987/     104  20423389.0  ...         0.0   
1  https://www.imdb.com/title/tt0098994/     114   2700000.0  ...         0.0   
2  https://www.imdb.com/title/tt0099005/     112  33461269.0  ...         0.0   
3  https://www.imdb.com/title/tt0099012/     102   7331647.0  ...         0.0   
4  https://www.imdb.com/title/tt0099018/      95   6939946.0  ...         0.0   

   rating_1.0  rating_1.5 rating_2.0 rating_2.5 rating_3.0 rating_3.5  \
0         0.0         0.0        0.0        0.0        0.0        0.0   
1         0.0         0.0        0.0        0.0        0.0        0.0   
2         0.0         0.0        0.0        0.0        0.0        0.0   
3         0.0         0.0        0.0        0.0        0.0        0.0   
4         0.0         0.0        0.0        0.0        0.0        0.0   

  rating_4.0 rating_4.5 rating_5.0  
0        0.0        0.0        0.0  
1        0.0        0.0        0.0  
2        0.0        0.0        0.0  
3        0.0        0.0        0.0  
4        0.0        0.0        0.0  

[5 rows x 37 columns]